In [1]:
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

## Importamos los archivos

In [2]:
df_bajas = pd.read_feather('../../../data/processed/periodos_baja_x_cif_id.feather')

In [3]:
df_pagos = pd.read_feather('../../../data/processed/pagos_x_cif_id.feather')

In [4]:
df_interacciones = pd.read_feather('../../../data/processed/interacciones_tipo_periodo_x_cif_id.feather')

In [5]:
df_clientes = pd.read_feather('../../../data/interim/clientes.feather')

In [6]:
df_clientes = df_clientes.rename(columns={"ID":"CIF_ID"})

In [7]:
df_siniestros = pd.read_feather('../../../data/processed/siniestros_x_cif_id.feather')

## Verificamos que todos se puedan combinar con la tabla Bajas

In [8]:
def check_merge(df1,df2,field):
    return len(set(df1[field].unique()).intersection(set(df2[field].unique()))) / len(df1[field].unique())

In [9]:
check_merge(df_bajas, df_pagos, "CIF_ID")

0.9920195773212915

In [10]:
check_merge(df_bajas, df_clientes, "CIF_ID")

0.9868415534564975

*Renombrando columnas de interacciones y descartando periodos que no vamos a analizar*

In [11]:
df_interacciones.columns

Index(['CIF_ID', '(1, 'A')', '(1, 'I')', '(1, 'O')', '(2, 'A')', '(2, 'I')',
       '(2, 'O')', '(3, 'A')', '(3, 'I')', '(3, 'O')', '(4, 'A')', '(4, 'I')',
       '(4, 'O')', '(5, 'A')', '(5, 'I')', '(5, 'O')', '(6, 'A')', '(6, 'I')',
       '(6, 'O')', '(7, 'A')', '(7, 'I')', '(7, 'O')', '(8, 'A')', '(8, 'I')',
       '(8, 'O')', '(9, 'A')', '(9, 'I')', '(9, 'O')', '(10, 'A')',
       '(10, 'I')', '(10, 'O')', '(11, 'A')', '(11, 'I')', '(11, 'O')',
       '(12, 'A')', '(12, 'I')', '(12, 'O')', '(13, 'A')', '(13, 'I')',
       '(13, 'O')', '(14, 'A')', '(14, 'I')', '(14, 'O')', '(15, 'A')',
       '(15, 'I')', '(15, 'O')', '(16, 'A')', '(16, 'I')', '(16, 'O')',
       '(17, 'A')', '(17, 'I')', '(17, 'O')', '(18, 'A')', '(18, 'I')',
       '(18, 'O')', '(19, 'A')', '(19, 'I')', '(19, 'O')', '(20, 'A')',
       '(20, 'I')', '(20, 'O')'],
      dtype='object')

In [12]:
df_interacciones = df_interacciones.rename(columns=lambda x: x.replace("(","").replace(")","").replace(", ","_TIPOINT_").replace("'","")) 

In [13]:
to_drop = [col for col in df_interacciones.drop('CIF_ID', axis=1).columns if int(col.split('_')[0])>18]

In [14]:
to_drop

['19_TIPOINT_A',
 '19_TIPOINT_I',
 '19_TIPOINT_O',
 '20_TIPOINT_A',
 '20_TIPOINT_I',
 '20_TIPOINT_O']

In [15]:
df_interacciones.drop(to_drop, axis=1, inplace=True)

In [16]:
check_merge(df_bajas, df_interacciones, "CIF_ID")

0.9998001810704867

In [17]:
check_merge(df_bajas, df_siniestros, "CIF_ID")

0.09662602037166321

## Merge entre tablas

In [18]:
len(df_bajas)

405367

In [19]:
len(df_interacciones)

1872729

In [20]:
len(df_pagos)

979749

In [21]:
len(df_clientes)

11650636

In [22]:
len(df_siniestros)

92314

In [23]:
df_bajas_int =  pd.merge(df_bajas, df_interacciones, on=['CIF_ID'], how='left')

In [24]:
df_bajas_int.head()

,CIF_ID,hist_polizas,periodo_baja,1_TIPOINT_A,1_TIPOINT_I,1_TIPOINT_O,2_TIPOINT_A,2_TIPOINT_I,2_TIPOINT_O,3_TIPOINT_A,3_TIPOINT_I,3_TIPOINT_O,4_TIPOINT_A,4_TIPOINT_I,4_TIPOINT_O,5_TIPOINT_A,5_TIPOINT_I,5_TIPOINT_O,6_TIPOINT_A,6_TIPOINT_I,6_TIPOINT_O,7_TIPOINT_A,7_TIPOINT_I,7_TIPOINT_O,8_TIPOINT_A,8_TIPOINT_I,8_TIPOINT_O,9_TIPOINT_A,9_TIPOINT_I,9_TIPOINT_O,10_TIPOINT_A,10_TIPOINT_I,10_TIPOINT_O,11_TIPOINT_A,11_TIPOINT_I,11_TIPOINT_O,12_TIPOINT_A,12_TIPOINT_I,12_TIPOINT_O,13_TIPOINT_A,13_TIPOINT_I,13_TIPOINT_O,14_TIPOINT_A,14_TIPOINT_I,14_TIPOINT_O,15_TIPOINT_A,15_TIPOINT_I,15_TIPOINT_O,16_TIPOINT_A,16_TIPOINT_I,16_TIPOINT_O,17_TIPOINT_A,17_TIPOINT_I,17_TIPOINT_O,18_TIPOINT_A,18_TIPOINT_I,18_TIPOINT_O
0,1313.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,NaN,1.0,2.0,NaN,NaN,2.0,NaN,4.0,2.0,NaN,1.0,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,1.0,3.0,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,5.0,NaN,NaN,2.0,NaN,NaN,6.0,NaN,NaN,2.0,NaN,1.0,2.0
1,1482.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,3.0
2,1559.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,NaN,8.0,1.0,NaN,4.0,3.0,NaN,6.0,3.0,NaN,5.0,3.0,NaN,2.0,1.0,NaN,3.0,4.0,NaN,5.0,6.0,NaN,1.0,3.0,NaN,2.0,2.0,1.0,9.0,6.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,1.0,8.0,NaN,NaN,2.0,NaN,4.0,9.0
3,1779.0,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,13.0,10.0,NaN,NaN,3.0,NaN,NaN,1.0,NaN,NaN,4.0,NaN,NaN,2.0,NaN,NaN,1.0
4,2994.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,3.0,NaN,1.0,1.0,NaN,5.0,7.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,2.0,NaN,1.0,1.0,NaN,NaN,2.0,NaN,1.0,NaN,NaN,1.0,2.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN


In [25]:
df_bajas_int_sini =  pd.merge(df_bajas_int, df_siniestros, on=['CIF_ID'], how='left')

In [36]:
df_bajas_int_sini = df_bajas_int_sini.fillna(0)

NameError: name 'df_bajas_sini' is not defined

In [26]:
df_bajas_int_sini.head()

,CIF_ID,hist_polizas,periodo_baja,1_TIPOINT_A,1_TIPOINT_I,1_TIPOINT_O,2_TIPOINT_A,2_TIPOINT_I,2_TIPOINT_O,3_TIPOINT_A,3_TIPOINT_I,3_TIPOINT_O,4_TIPOINT_A,4_TIPOINT_I,4_TIPOINT_O,5_TIPOINT_A,5_TIPOINT_I,5_TIPOINT_O,6_TIPOINT_A,6_TIPOINT_I,6_TIPOINT_O,7_TIPOINT_A,7_TIPOINT_I,7_TIPOINT_O,8_TIPOINT_A,8_TIPOINT_I,8_TIPOINT_O,9_TIPOINT_A,9_TIPOINT_I,9_TIPOINT_O,10_TIPOINT_A,10_TIPOINT_I,10_TIPOINT_O,11_TIPOINT_A,11_TIPOINT_I,11_TIPOINT_O,12_TIPOINT_A,12_TIPOINT_I,12_TIPOINT_O,13_TIPOINT_A,13_TIPOINT_I,13_TIPOINT_O,14_TIPOINT_A,14_TIPOINT_I,14_TIPOINT_O,15_TIPOINT_A,15_TIPOINT_I,15_TIPOINT_O,16_TIPOINT_A,16_TIPOINT_I,16_TIPOINT_O,17_TIPOINT_A,17_TIPOINT_I,17_TIPOINT_O,18_TIPOINT_A,18_TIPOINT_I,18_TIPOINT_O,periodo_sini_1,periodo_sini_2,periodo_sini_3,periodo_sini_4,periodo_sini_5,periodo_sini_6,periodo_sini_7,periodo_sini_8,periodo_sini_9,periodo_sini_10,periodo_sini_11,periodo_sini_12,periodo_sini_13,periodo_sini_14,periodo_sini_15,periodo_sini_16,periodo_liquidacion_sini_1,periodo_liquidacion_sini_2,periodo_liquidacion_sini_3,periodo_liquidacion_sini_4,periodo_liquidacion_sini_5,periodo_liquidacion_sini_6,periodo_liquidacion_sini_7,periodo_liquidacion_sini_8,periodo_liquidacion_sini_9,periodo_liquidacion_sini_10,periodo_liquidacion_sini_11,periodo_liquidacion_sini_12,periodo_liquidacion_sini_13,periodo_liquidacion_sini_14,periodo_liquidacion_sini_15,periodo_liquidacion_sini_16,periodo_rechazo_sini_1,periodo_rechazo_sini_2,periodo_rechazo_sini_3,periodo_rechazo_sini_4,periodo_rechazo_sini_5,periodo_rechazo_sini_6,periodo_rechazo_sini_7,periodo_rechazo_sini_8,periodo_rechazo_sini_9,periodo_rechazo_sini_10,periodo_rechazo_sini_11,periodo_rechazo_sini_12,periodo_rechazo_sini_13,periodo_rechazo_sini_14,periodo_rechazo_sini_15,periodo_rechazo_sini_16,index,1_TIPO_SINI_A,1_TIPO_SINI_B,1_TIPO_SINI_C,1_TIPO_SINI_I,1_TIPO_SINI_N,2_TIPO_SINI_A,2_TIPO_SINI_B,2_TIPO_SINI_C,2_TIPO_SINI_I,2_TIPO_SINI_N,3_TIPO_SINI_A,3_TIPO_SINI_B,3_TIPO_SINI_C,3_TIPO_SINI_I,3_TIPO_SINI_N,4_TIPO_SINI_A,4_TIPO_SINI_B,4_TIPO_SINI_C,4_TIPO_SINI_I,4_TIPO_SINI_N,5_TIPO_SINI_A,5_TIPO_SINI_B,5_TIPO_SINI_C,5_TIPO_SINI_I,5_TIPO_SINI_L,5_TIPO_SINI_N,6_TIPO_SINI_A,6_TIPO_SINI_B,6_TIPO_SINI_C,6_TIPO_SINI_I,6_TIPO_SINI_N,7_TIPO_SINI_A,7_TIPO_SINI_B,7_TIPO_SINI_C,7_TIPO_SINI_I,7_TIPO_SINI_N,8_TIPO_SINI_A,8_TIPO_SINI_B,8_TIPO_SINI_C,8_TIPO_SINI_I,8_TIPO_SINI_N,9_TIPO_SINI_A,9_TIPO_SINI_B,9_TIPO_SINI_C,9_TIPO_SINI_I,9_TIPO_SINI_N,10_TIPO_SINI_A,10_TIPO_SINI_B,10_TIPO_SINI_C,10_TIPO_SINI_I,10_TIPO_SINI_L,10_TIPO_SINI_N,11_TIPO_SINI_A,11_TIPO_SINI_B,11_TIPO_SINI_C,11_TIPO_SINI_E,11_TIPO_SINI_H,11_TIPO_SINI_I,11_TIPO_SINI_N,11_TIPO_SINI_Z,12_TIPO_SINI_A,12_TIPO_SINI_B,12_TIPO_SINI_C,12_TIPO_SINI_H,12_TIPO_SINI_I,12_TIPO_SINI_L,12_TIPO_SINI_N,13_TIPO_SINI_A,13_TIPO_SINI_B,13_TIPO_SINI_C,13_TIPO_SINI_E,13_TIPO_SINI_H,13_TIPO_SINI_I,13_TIPO_SINI_L,13_TIPO_SINI_N,13_TIPO_SINI_R,14_TIPO_SINI_A,14_TIPO_SINI_B,14_TIPO_SINI_C,14_TIPO_SINI_E,14_TIPO_SINI_H,14_TIPO_SINI_I,14_TIPO_SINI_L,14_TIPO_SINI_N,14_TIPO_SINI_Z,15_TIPO_SINI_A,15_TIPO_SINI_B,15_TIPO_SINI_C,15_TIPO_SINI_E,15_TIPO_SINI_H,15_TIPO_SINI_I,15_TIPO_SINI_L,15_TIPO_SINI_N,15_TIPO_SINI_R,15_TIPO_SINI_Z,16_TIPO_SINI_A,16_TIPO_SINI_B,16_TIPO_SINI_C,16_TIPO_SINI_E,16_TIPO_SINI_H,16_TIPO_SINI_I,16_TIPO_SINI_L,16_TIPO_SINI_N,16_TIPO_SINI_R,16_TIPO_SINI_Z
0,1313.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,NaN,1.0,2.0,NaN,NaN,2.0,NaN,4.0,2.0,NaN,1.0,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,1.0,3.0,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,5.0,NaN,NaN,2.0,NaN,NaN,6.0,NaN,NaN,2.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [30]:
from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left, right, on=['CIF_ID'], how='inner'), [df_bajas_int_sini, df_pagos, df_clientes])

In [31]:
df_merged.shape

(396835, 419)

### Guardamos la tabla completa

In [28]:
df_merged.to_feather('../../../data/processed/merged_data_18_periods.feather')

--- 

In [1]:
import pandas as pd
df_merged = pd.read_feather('../../../data/processed/merged_data_18_periods.feather')

In [34]:
"], [".join(set([''.join([s for s in col if not s.isdigit()]) for col in df_merged.columns]))

'_COBRO_CC], [periodo_liquidacion_sini_], [_TIPO_SINI_I], [_TIPO_SINI_R], [ANO_DE_NACIMIENTO], [_SITUACION_AM], [demora_pago_], [_TIPO_SINI_H], [ORIGEN], [POSICION_IVA], [_COBRO_SJ], [_COBRO_BA], [DATMAR_NIVEL_EDUCACION], [_COBRO_PP], [DATECO_RAZON_SOCIAL_EMPLEADOR], [DATECO_AUTONOMO], [TIPO_PERSONA], [_TIPO_SINI_C], [periodo_rechazo_sini_], [_TIPO_SINI_Z], [SEXO], [FECHA_NACIMIENTO], [_SITUACION_EP], [MONTO_PAGO_], [_TIPO_SINI_A], [CIF_ID], [_TIPO_SINI_L], [CLIENTE], [ESTADO_CIVIL], [DATECO_TRABAJA], [hist_polizas], [_TIPO_SINI_B], [periodo_baja], [_SITUACION_PP], [periodo_sini_], [_SITUACION_CT], [_COBRO_TA], [_TIPO_SINI_E], [_TIPOINT_A], [index], [_TIPOINT_I], [_TIPOINT_O], [DATMAR_COMPLETO_ESTUDIOS], [_TIPO_SINI_N'